In [ ]:
import pandas as pd
import numpy as np
from pylab import mpl, plt

raw=pd.read_csv('./OneMonthData.csv', index_col=0, parse_dates=True).dropna()
data=pd.DataFrame(raw[['time','close']][0:5000])
data['returns']=np.log(data['close']/data['close'].shift(1))
window=20
data['vol']=data['returns'].rolling(window).std()
data['mom']=data['returns'].rolling(window).mean()
data['sma']=data['close'].rolling(window).mean()
data['min']=data['close'].rolling(window).min()
data['max']=data['close'].rolling(window).max()
data.dropna(inplace=True)

lags=5
features=['returns', 'vol', 'mom', 'sma', 'min', 'max']
dataLags=np.lib.stride_tricks.sliding_window_view(data[features], window_shape=lags, axis=0)[:-1,:].reshape((-1, len(features)*lags))
data['direction']=np.where(data['returns']>0, 1, -1)
data.dropna(inplace=True)

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

n_estimators=15
random_state=100
max_depth=3
min_samples_leaf=15
subsample=0.33

tree=DecisionTreeClassifier(random_state=random_state, max_depth=max_depth, min_samples_leaf=min_samples_leaf)
model=AdaBoostClassifier(base_estimator=tree, n_estimators=n_estimators, random_state=random_state)

mu=np.mean(dataLags, axis=0)
std=np.std(dataLags, axis=0)
normalisedData=(dataLags-mu)/std

model.fit(normalisedData, data['direction'][lags:])


In [ ]:
import numpy as np
import pandas as pd
from pylab import mpl, plt
plt.style.use('seaborn')
mpl.rcParams['font.family'] = 'serif'

'''
Base class for the event-based testing of the trading strategies

Attributes
------------------------------
column : column to be used for trading
start  : starting date for the collection of data for trading
end    : ending date for the collection of data for trading
amount : amount to be traded 
ftc    : fixed transaction costs
ptc    : proportional transaction costs

Functions
-----------------------------------
get_data()         : retrieves the data for implementing the trading strategy
plot_data()        : plots the retrieved data
get_price()        : returns the price for the corresponding bar
print_balance()    : prints the current balance left
print_net_wealth() : prints the current net wealth
place_buy_order()  : places an order to buy foreign exchange
place_sell_order() : places an order to sell foreign exchange
close_out()        : closes the implementation of the trade
'''
class BacktestBase(object):
    def __init__(self, amount, ftc=0.0, ptc=0.0, verbose=True):
        self.initialAmount=amount    #stores the inital ammount allotted for trading (remains constant)
        self.amount=amount           #trading cash balance (running balance)
        self.ftc=ftc                 #fixed transaction costs per trade
        self.ptc=ptc                 #propertional transaction costs per trade
        self.units=0                 #number of units traded
        self.position=0              #initial position is set to neutral
        self.trades=0                #no trades are executed initially
        self.verbose=verbose         #prints detailed summary in terminal

    def set_prices(self, price):
        '''
        Keep track of prices to track the performance
        '''
        self.entry_price=price
        self.min_price=price
        self.max_price=price
        
    def print_balance(self, time):
        '''
        For the requested bar, prints the available balance
        '''
        print(f'{time} | current balance {self.amount:.2f}')

    def print_net_wealth(self, time, price):
        '''
        For the requested bar, prints the net wealth
        '''
        net_wealth=self.units*price+self.amount
        print(f'{time} | current net wealth {net_wealth:.2f}')
        
    def place_buy_order(self, time, price, units=None, amount=None):
        '''
        Place an order to buy foreign exchange for the passed amount or the passed number of units at the corresponding bar
        '''
        if units is None:
            units=int(amount/price)
        self.amount-=(units*price)*(1+self.ptc)+self.ftc
        self.units+=units
        self.trades+=1
        self.set_prices(price)
        if self.verbose==True:
            print(f'{time} | buying {units} units at {price:.3f}')
            self.print_balance(time)
            self.print_net_wealth(time, price)
            
    def place_sell_order(self, time, price, units=None, amount=None):
        '''
        Place an order to sell foreign exchange for the passed amount or the passed number of units at the corresponding bar
        '''
        if units is None:
            units=int(amount/price)
        self.amount+=(units*price)*(1-self.ptc)-self.ftc
        self.units-=units
        self.trades+=1
        self.set_prices(price)
        if self.verbose==True:
            print(f'{time} | selling {units} units at {price:.3f}')
            self.print_balance(time)
            self.print_net_wealth(time, price)
            
    def close_out(self, time, price):
        '''
        Closes out any remaining open position at the corresponding bar 
        '''
        #self.amount+=self.units*price
        if self.units>0:
            self.place_sell_order(time, price, units=self.units)
        print(50*'_')
        print(f'{time} | --- CLOSING OUT ---')
        if self.verbose==True:
            print(f'{time} | remianing {self.units} units at {price:.3f}')
            print(50*'-')
        print('Final balance [$] {:.3f}'.format(self.amount))
        perf = ((self.amount-self.initialAmount)/self.initialAmount*100)
        print('Net Performance [%] {:.3f}'.format(perf))
        print('Trades Executed [#] {:.3f}'.format(self.trades))
        print(50*'_')

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots

frame=make_subplots(rows=2, cols=1, shared_xaxes=True, subplot_titles=("EUR_USD EXC Rate", "Returns"))
frame.append_trace(go.Scatter(name='EXCHANGE RATE'), row=1, col=1)
frame.append_trace(go.Scatter(name='returns'), row=2, col=1)
frame.append_trace(go.Scatter(name='momentum'), row=2, col=1)
frame.update_layout(height=800)

fig=go.FigureWidget(frame)
fig


In [ ]:
import numpy as np
import pandas as pd
import zmq
import datetime
import MetaTrader5 as mt5
import time

class BacktestLongOnly(BacktestBase):
    
    def get_prediction(self, data):
        calc_data=pd.DataFrame(data[-(window+lags+1):])
        calc_data['returns']=np.log(calc_data/calc_data.shift(1))
        calc_data['vol']=calc_data['returns'].rolling(window).std()
        calc_data['mom']=calc_data['returns'].rolling(window).mean()
        calc_data['sma']=calc_data['bid'].rolling(window).mean()
        calc_data['min']=calc_data['bid'].rolling(window).min()
        calc_data['max']=calc_data['bid'].rolling(window).max()
        fig.data[0].x=self.times
        fig.data[0].y=calc_data['bid']
        fig.data[1].x=self.times
        fig.data[1].y=calc_data['returns']
        fig.data[2].x=self.times
        fig.data[2].y=calc_data['mom']
        self.times.pop(0)
        calc_data.dropna(inplace=True)
        features=['returns', 'vol', 'mom', 'sma', 'min', 'max']
        feature_vector=np.lib.stride_tricks.sliding_window_view(calc_data[features], window_shape=lags, axis=0)[:-1,:].reshape((-1, len(features)*lags))
        feature_vector=(feature_vector-mu)/std
        action=model.predict(feature_vector)
        if action==1:
            fig.add_vrect(
                x0=self.times[-1], x1=self.times[-2],
                fillcolor="green", opacity=0.5,
                layer="below", line_width=0,
            )
        if action==-1:
            fig.add_vrect(
                x0=self.times[-1], x1=self.times[-2],
                fillcolor="red", opacity=0.5,
                layer="below", line_width=0,
            )
        return action
        
    def run_testing(self, window=20, lags=10):
        
        # display data on the MetaTrader 5 package
        print("MetaTrader5 package author: ",mt5.__author__)
        print("MetaTrader5 package version: ",mt5.__version__)
        
        # establish connection to the MetaTrader 5 terminal
        if not mt5.initialize():
            print("initialize() failed, error code =",mt5.last_error())
            quit()
        
        print("Connection to MT5 server established...")
        
        # attempt to enable the display of the EURUSD in MarketWatch
        selected=mt5.symbol_select("EURUSD",True)
        if not selected:
            print("Failed to select EURUSD")
            mt5.shutdown()
            quit()
    
        data=pd.DataFrame()        
        
        text=f'\n\nRunning Decision Tree based strategy (online) | lags={lags}'
        text+=f'\nfixed costs {self.ftc} | '
        text+=f'proportional costs {self.ptc}'
        print(text)          #print the parameters
        print('='*55)
        self.units=0
        self.position=0      #initially the position is neutral
        self.trades=0        #no trades have been exeecuted yet
        self.amount=self.initialAmount  #reset initial capital
        self.sl=0.0001
        self.tsl=0.0001
        self.tp=0.0010
        tickval=0
        self.times=list()
        
        #get and display the last EURUSD tick at interval of 30 seconds
        while tickval<10000:
            lasttick=mt5.symbol_info_tick("EURUSD")
            data=data.append(lasttick._asdict(), ignore_index=True)
            lasttick=lasttick._asdict()
            val=lasttick['bid']
            #val=data['bid'][-1]
            t=datetime.datetime.now()
            self.times.append(t)
            
            if len(data)>window+lags:
                reqData=pd.DataFrame(data['bid'][-(window+lags+1):])
                if self.trades==0:
                    print(50*'_')
                    print(f'{tickval} | --- START TESING ---')
                    print(50*'-') 
                if self.sl!=0 and self.position!=0:
                    change=(val-self.entry_price)/self.entry_price
                    if self.position==1 and change<-self.sl:
                        print(50*'-')
                        print('--- STOP LOSS (LONG | -{self.sl}) ---')
                        self.place_buy_order(time=t, price=val, units=self.units)
                        self.position=-1
                    elif self.position==-1 and change>self.sl:
                        print(50*'-')
                        print('--- STOP LOSS (SHORT | -{self.sl}) ---')
                        self.place_sell_order(time=t, price=val, units=self.units)
                        self.position=1
                if self.tsl!=0 and self.position!=0:
                    self.max_price=max(self.max_price, val)
                    self.min_price=min(self.min_price, val)
                    if self.position==1 and (val-self.max_price)/self.entry_price<-self.tsl:
                        print(50*'-')
                        print('--- TRAILING STOP LOSS (LONG | -{self.tsl}) ---')
                        self.place_buy_order(time=t, price=val, units=self.units)
                        self.position=-1
                    elif self.position==-1 and (self.min_price-val)/self.entry_price<-self.tsl:
                        print(50*'-')
                        print('--- TRAILING STOP LOSS (SHORT | -{self.tsl}) ---')
                        self.place_sell_order(time=t, price=val, units=self.units)
                        self.position=1
                if self.tp!=0 and self.position!=0:
                    change=(val-self.entry_price)/self.entry_price
                    if self.position==1 and change>self.tp:
                        print(50*'-')
                        print('--- TAKE PROFIT (LONG | -{self.sl}) ---')
                        self.place_buy_order(time=t, price=val, units=self.units)
                        self.position=-1
                    elif self.position==-1 and change<-self.tp:
                        print(50*'-')
                        print('--- TAKE PROFIT (SHORT | -{self.sl}) ---')
                        self.place_sell_order(time=t, price=val, units=self.units)
                        self.position=1
                action=self.get_prediction(reqData)                
                if self.position==-1 or self.position==0:
                    if action==1:
                        print(50*'_')
                        print(f'{tickval} | --- GO LONG ---')
                        self.place_buy_order(time=t, price=val, amount=self.amount)
                        self.position=1  # long position
                elif self.position==1:
                    if action==-1:
                        print(50*'_')
                        print(f'{tickval} | --- GO SHORT ---')
                        self.place_sell_order(time=t, price=val, units=self.units)
                        self.position=0  # market neutral
            tickval+=1
            time.sleep(30)
        # shut down connection to the MetaTrader 5 terminal
        mt5.shutdown()
        self.close_out(time=t, price=val)
        


In [ ]:
if __name__ == '__main__':
    test_long=BacktestLongOnly(amount=1000, verbose=True)
    test_long.run_testing()


In [ ]:
import MetaTrader5 as mt5
import time
import pandas as pd
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
 
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()
 
# attempt to enable the display of the EURUSD in MarketWatch
selected=mt5.symbol_select("EURUSD",True)
if not selected:
    print("Failed to select EURUSD")
    mt5.shutdown()
    quit()
    
data=pd.DataFrame()
 
#get and display the last GBPUSD tick at interval of 30 seconds
for i in range(50):
    lasttick=mt5.symbol_info_tick("GBPUSD")
    print(lasttick)
    data=data.append(lasttick._asdict(), ignore_index=True)
    # display tick field values as the last entry of pandas dataframe
    print(data.tail(n=1))
    time.sleep(30)
 
# shut down connection to the MetaTrader 5 terminal
mt5.shutdown()

In [ ]:
import MetaTrader5 as mt5
import time
import pandas as pd
# display data on the MetaTrader 5 package
print("MetaTrader5 package author: ",mt5.__author__)
print("MetaTrader5 package version: ",mt5.__version__)
 
# establish connection to the MetaTrader 5 terminal
if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

# now connect to another trading account specifying the password
account = #########
authorized = mt5.login(account, password="#########", server="MetaQuotes-Demo")

if authorized:
    # attempt to enable the display of the GBPUSD in MarketWatch
    selected=mt5.symbol_select("GBPUSD",True)
    if not selected:
        print("Failed to select GBPUSD")
        mt5.shutdown()
        quit()
    
    data=pd.DataFrame()
 
    #get and display the last GBPUSD tick at interval of 30 seconds
    for i in range(50):
        lasttick=mt5.symbol_info_tick("GBPUSD")
        print(lasttick)
        data=data.append(lasttick._asdict(), ignore_index=True)
        # display tick field values as the last entry of pandas dataframe
        print(data.tail(n=1))
        time.sleep(30)
 
# shut down connection to the MetaTrader 5 terminal
mt5.shutdown()